# Without Feature Engineering with isDebt Column

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier

In [49]:
df=pd.read_csv('./Dataset/bank_cleansed.csv')
df

,age,default,balance,housing,loan,campaign,pdays,previous,deposit,month_num,...,categorical__month_jun,categorical__month_mar,categorical__month_may,categorical__month_nov,categorical__month_oct,categorical__month_sep,categorical__poutcome_failure,categorical__poutcome_other,categorical__poutcome_success,categorical__poutcome_unknown
0,59,0,2343,1,0,1,-1,0,1,5,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,56,0,45,0,0,1,-1,0,1,5,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,41,0,1270,1,0,1,-1,0,1,5,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,55,0,2476,1,0,1,-1,0,1,5,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,54,0,184,0,0,2,-1,0,1,5,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,33,0,1,1,0,1,-1,0,0,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
11158,39,0,733,0,0,4,-1,0,0,6,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
11159,32,0,29,0,0,2,-1,0,0,8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
11160,43,0,0,0,1,2,172,5,0,5,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [50]:
seed=67
X=df.drop(columns='deposit')
y=df['deposit']
X_train,X_test,y_train,y_test=train_test_split(X,y,stratify=y,random_state=seed,test_size=0.2)

In [ ]:
import pickle
xgb=XGBClassifier()
params_grid = param_grid = {
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.05, 0.1],
    'n_estimators': [200, 300],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'min_child_weight': [1, 3],
    'gamma': [0, 0.1]
}
gs=GridSearchCV(estimator=xgb,param_grid=params_grid,scoring='roc_auc')
gs.fit(X_train,y_train)
best_model_without_feature_en=gs.best_estimator_
with open("./Models/model_without_feature_en.pkl", "wb") as f:
    pickle.dump(best_model_without_feature_en, f)

NameError: name 'XGBClassifier' is not defined

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report,roc_auc_score
pred=best_model_without_feature_en.predict(X_test)
y_proba=best_model_without_feature_en.predict_proba(X_test)[:,1]
print(confusion_matrix(y_true=y_test,y_pred=pred))
print(classification_report(y_true=y_test,y_pred=pred))
print(roc_auc_score(y_test,y_proba))

[[985 190]
 [429 629]]
              precision    recall  f1-score   support

           0       0.70      0.84      0.76      1175
           1       0.77      0.59      0.67      1058

    accuracy                           0.72      2233
   macro avg       0.73      0.72      0.72      2233
weighted avg       0.73      0.72      0.72      2233

0.7729236214455215
